# Retrieval Augmented Generation

## Setup API clients

In [2]:
import os

import azure.identity
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

azure_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

AZURE_SEARCH_FULL_INDEX = "purple-cat-jt3t703gbc"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=azure_credential)


## Prepare user question

In [3]:
user_question = "Who is Prateek?"
user_question_vector = get_embedding(user_question)

## Retrieve matching documents

The search call below does a **hybrid search**, performing both a full-text search and a vector search in parallel.
It merges those results using Reciprocal Rank Fusion (RRF). 
Finally, it re-ranks the merged results using the AI Search semantic ranker, a re-ranking model.

In [12]:
# Perform a keyword search
keyword_query = "prateek"
results = search_client.search(
    search_text=keyword_query,  # The keyword or phrase to search for
    top=5  # Number of results to retrieve
)

# Process and print the results
for result in results:
    print(f"Source: {result['sourcepage']}")
    print(f"Content: {result['content']}\n")

HttpResponseError: Operation returned an invalid status 'Forbidden'

## Send question and documents to the model

In [4]:
SYSTEM_MESSAGE = """
Assistant helps company employees questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
USER_MESSAGE = user_question + "\nSources: " + sources

# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

A product manager is responsible for overseeing the product management team, driving product development and marketing strategy, and ensuring successful product execution [role_library-28.png]. They collaborate with internal teams such as engineering, sales, marketing, and finance, as well as external partners, suppliers, and customers [role_library-28.png]. They develop and implement product life-cycle management processes, monitor industry trends, develop product marketing plans and go-to-market strategies, research customer needs, collaborate with internal teams, develop pricing strategies and cost models, oversee product portfolio and performance metrics, manage product development budget, and analyze product performance and customer feedback [role_library-28.png]. They should have a bachelor's degree in business, engineering, or a related field and at least 5 years of experience in product management with a proven track record of successful product launches [role_library-28.png]. 